# Snowpark Sessions

A **session** is an entry point for the DataFrame API. It holds a connection to a Snowflake instance. A session can be initialized by providing a config file or a Map of config properties. You can set any parameters supported by the JDBC Driver. (For a list of these parameters, see [Connection Parameters](https://docs.snowflake.com/en/user-guide/jdbc-configure.html#connection-parameters) in the JDBC driver documentation.)

In this Notebook you create a connection properties file to define the Snowpark connection parameters, and this will also be used in the following labs.

---

## Initialize Account and User parameters 

Update the variable values below with your Snowflake account name, user name and password.

In [ ]:
val acct    = "<your_snowflake_account_name>" // account
val user    = "<your_snowflake_username>"     // user
val pw      = new Password("PASSWORD_YOU_SET_IN_SNOWFLAKE") // password

## Generate a Connection Properties File


```
# connect.properties file (a text file)
URL = https://<account>.snowflakecomputing.com
USER = <username>
PASSWORD = <secret>
...
```

Any additional JDBC parameters (see [Connection Parameters](https://docs.snowflake.com/en/user-guide/jdbc-configure.html#connection-parameters) in the JDBC driver documentation) needed to connect to Snowflake can be specified (e.g. USER, ROLE, WAREHOUSE, DB, SCHEMA, etc.).

Using the locally defined variables, generate a properties file that contains the session properties and pass it to the session.
Note: you do not have to add anything to the following code block, simply run it to generate the config file.

In [ ]:
import java.io._

// Create a Map to store the connection properties
var connectionMap = Map(
    "URL" -> s"https://$acct.snowflakecomputing.com",
    "USER" -> user,
    "PASSWORD" -> pw.value,
    "WAREHOUSE" -> user.concat("_wh"),
    "DB" -> user.concat("_db"),
    "SCHEMA" -> "raw"
    )

// Write the properties key value pairs to a file
val pwd = sys.env.get("PWD").fold("")(_.toString)
val filename = s"$pwd/de_snowpark/connect.properties"
val file = new File(filename)
val bw = new BufferedWriter(new FileWriter(file))

bw.write("# connect.properties file (a text file)\n")

connectionMap.keys.foreach{ i => 
    val value = connectionMap.get(i).fold("")(_.toString)
    bw.write( s"$i = $value\n") }

bw.close()


---
## Create a Session 

Create a SessionBuilder object and pass in the connection properties filename and create a Snowpark session.

In [ ]:
import com.snowflake.snowpark._

val session = Session.builder.configFile(s"$filename").create

---
## Get Session Values

Use the session object to get the current database, schema and query tag values.


In [ ]:
var currentDb = session.getCurrentDatabase.fold("")(_.toString)
println(s"The current database is $currentDb")

var currentSchema = session.getCurrentSchema.fold("")(_.toString)
println(s"The current schema is $currentSchema")

var currentTag = session.getQueryTag
println(s"The current query tag is $currentTag")

---
## Define a Session Query Tag

Define a query tag variable and generate the query tag value.

Set the session query tag.

In [ ]:
var userQueryTag = "SNOWPARK_".concat( sys.env.get("JUPYTERHUB_USER") .fold("")(_.toString) .toUpperCase())

session.setQueryTag(userQueryTag)

var currentTag = session.getQueryTag.fold("")(_.toString)
println(s"The current query tag is $currentTag")